# Database Admin 101 - Lab

## Introduction 

In this lab, you'll go through the process of designing and creating a database. From there, you'll begin to populate this table with mock data provided to you.

## Objectives

You will be able to:

* Use knowledge of the structure of databases to create a database and populate it

## The Scenario

You are looking to design a database for a school that will house various information from student grades to contact information, class roster lists and attendance. First, think of how you would design such a database. What tables would you include? What columns would each table have? What would be the primary means to join said tables?

## Creating the Database

Now that you've put a little thought into how you might design your database, it's time to go ahead and create it! Start by import the necessary packages. Then, create a database called **school.sqlite**.

In [4]:
# Import necessary packages
import sqlite3
import pandas as pd

In [5]:
#Create the database school.sqlite 
conn = sqlite3.connect('school.sqlite')
c = conn.cursor()

## Create a Table for Contact Information

Create a table called contactInfo to house contact information for both students and staff. Be sure to include columns for first name, last name, role (student/staff), telephone number, street, city, state, and zipcode. Be sure to also create a primary key for the table. 

In [7]:
# Your code here
c.execute('''
CREATE TABLE StudentInfo(
id INT PRIMARY KEY,
firstname TEXT,
lastname TEXT,
role TEXT,
tel_num INT,
street TEXT,
city TEXT,
state TEXT,
zipcode TEXT);
''')

## Populate the Table

Below, code is provided for you in order to load a list of dictionaries. Briefly examine the list. Each dictionary in the list will serve as an entry for your contact info table. Once you've briefly investigated the structure of this data, write a for loop to iterate through the list and create an entry in your table for each person's contact info.

In [10]:
# Load the list of dictionaries; just run this cell
import pickle

with open('contact_list.pickle', 'rb') as f:
    contacts = pickle.load(f)
contacts[0]

{'firstName': 'Christine',
 'lastName': 'Holden',
 'role': 'staff',
 'telephone ': 2035687697,
 'street': '1672 Whitman Court',
 'city': 'Stamford',
 'state': 'CT',
 'zipcode ': '06995'}

In [14]:
# Iterate over the contact list and populate the contactInfo table here
for contact in contacts:
    firstname = contact['firstName']
    lastname = contact['lastName']
    role = contact['role']
    tel_num = contact['telephone ']
    street = contact['street']
    city = contact['city']
    state = contact['state']
    zipcode = contact['zipcode ']
    c.execute('''
    INSERT INTO StudentInfo (firstname, lastname,role,tel_num, street, city, state, zipcode)
    VALUES ('{}','{}','{}','{}','{}','{}','{}','{}');
    '''.format(firstname, lastname,role,tel_num, street, city, state, zipcode))



**Query the Table to Ensure it is populated**

In [15]:
# Your code here 
c.execute('''
SELECT *
FROM StudentInfo;''')
df = pd.DataFrame(c.fetchall())
df.columns = [i[0]for i in c.description]
df

,id,firstname,lastname,role,tel_num,street,city,state,zipcode
0,None,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,06995
1,None,Christopher,Warren,student,2175150957,1935 University Hill Road,Champaign,IL,61938
2,None,Linda,Jacobson,staff,4049446441,479 Musgrave Street,Atlanta,GA,30303
3,None,Andrew,Stepp,student,7866419252,2981 Lamberts Branch Road,Hialeah,Fl,33012
4,None,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
5,None,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
6,None,Mary,Raines,student,9075772295,3975 Jerry Toth Drive,Ninilchik,AK,99639
7,None,Ed,Lyman,student,5179695576,3478 Be Sreet,Lansing,MI,48933


## Commit Your Changes to the Database

Persist your changes by committing them to the database.

In [16]:
# Your code here
conn.commit()

## Create a Table for Student Grades

Create a new table in the database called "grades". In the table, include the following fields: userId, courseId, grade.

** This problem is a bit more tricky and will require a dual key. (A nuance you have yet to see.)
Here's how to do that:

```SQL
CREATE TABLE table_name(
   column_1 INTEGER NOT NULL,
   column_2 INTEGER NOT NULL,
   ...
   PRIMARY KEY(column_1,column_2,...)
);
```

In [17]:
# Create the grades table
c.execute('''
CREATE TABLE grades(
   userId INTEGER NOT NULL,
   courseId INTEGER NOT NULL,
   grade INTEGER, 
   PRIMARY KEY(userId,courseId));
   ''')

## Remove Duplicate Entries

An analyst just realized that there is a duplicate entry in the contactInfo table! Find and remove it.

In [18]:
# Find the duplicate entry
c.execute('''
SELECT firstname, lastname,role,tel_num, street, city, COUNT(*)
FROM StudentInfo
GROUP BY firstname, lastname,tel_num
HAVING COUNT(*) > 1;''').fetchall()

[('Jane', 'Evans', 'student', 3259909290, '1461 Briarhill Lane', 'Abilene', 2)]

In [19]:
# Delete the duplicate entry
c.execute('''
DELETE FROM StudentInfo
WHERE tel_num = 3259909290;
''')

In [22]:
# Check that the duplicate entry was removed
c.execute('''
SELECT firstname, lastname,role,tel_num, street, city, COUNT(*)
FROM StudentInfo
GROUP BY firstname, lastname,tel_num
HAVING COUNT(*) > 1;''').fetchall()

[]

In [23]:
conn.commit()

In [25]:
c.execute('''
SELECT firstname, lastname,role,tel_num, street, city, COUNT(*)
FROM StudentInfo
WHERE tel_num = 3259909290;
''').fetchall()

[(None, None, None, None, None, None, 0)]

## Updating an Address

Ed Lyman just moved to `2910 Simpson Avenue York, PA 17403`. Update his address accordingly.

In [29]:
# Update Ed's address
c.execute('''
UPDATE StudentInfo
SET street = '2910 Simpson Avenue',
     city = 'York', state = 'PA', zipcode = '17403' 
WHERE firstname = 'Ed'AND lastname = 'Lyman';
''')

In [30]:
# Query the database to ensure the change was made
c.execute('''
SELECT *
FROM StudentInfo;''')
df = pd.DataFrame(c.fetchall())
df.columns = [i[0]for i in c.description]
df

,id,firstname,lastname,role,tel_num,street,city,state,zipcode
0,None,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,06995
1,None,Christopher,Warren,student,2175150957,1935 University Hill Road,Champaign,IL,61938
2,None,Linda,Jacobson,staff,4049446441,479 Musgrave Street,Atlanta,GA,30303
3,None,Andrew,Stepp,student,7866419252,2981 Lamberts Branch Road,Hialeah,Fl,33012
4,None,Mary,Raines,student,9075772295,3975 Jerry Toth Drive,Ninilchik,AK,99639
5,None,Ed,Lyman,student,5179695576,2910 Simpson Avenue,York,PA,17403


## Commit Your Changes to the Database

Once again, persist your changes by committing them to the database.

In [31]:
# Your code here
conn.commit()

## Summary

While there's certainly more to do with setting up and managing this database, you got a taste for creating, populating, and maintaining databases! Feel free to continue fleshing out this exercise for more practice. 